## Video Redaction CPU
Here we are reading in a video file, identifying the face, drawing a rectangle around the face and blurring the area inside the rectangle
For the ease of working with your Nano remotely within Jupyter-lab, we are reducing the size of the video and displaying it in a widget.

We are using the magic %%time to view the overall run of the code block

In [3]:
%%time
#creative common video downloaded from https://www.youtube.com/watch?v=BLaf9_jHQyQ

import ipywidgets
from IPython.display import display
import numpy as np
import base64
import cv2

vwidth = 640
vheight = 480 

image_widget = ipywidgets.Image(format='jpeg')

display(image_widget)

cap = cv2.VideoCapture('faceR.mp4')

frames = []

fpath='./cascades/haarcascade_frontalface_alt.xml'

face_detect = cv2.CascadeClassifier(fpath)

while(1):
    try:
        _, frame = cap.read()

        frame_resized = cv2.resize(frame,(int(vwidth),int(vheight)))
        imgframe = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(imgframe, cv2.COLOR_BGR2GRAY)
        
        faces = face_detect.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(frame_resized,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame_resized[y:y+h, x:x+w]
            roi = cv2.GaussianBlur(roi_color, (23, 23), 30)
            # impose this blurred image on original image to get final image
            frame_resized[y:y+roi.shape[0], x:x+roi.shape[1]] = roi
        image_widget.value = bytes(cv2.imencode('.jpg', frame_resized)[1])
      
    except Exception:
        break

cap.release()



Image(value=b'', format='jpeg')

CPU times: user 1min 14s, sys: 2.89 s, total: 1min 16s
Wall time: 23.8 s


## Video Redaction GPU
Now lets use the GPU to performa the image classification.
Did this code block run faster or slower than the previous code block?

In [8]:
%%time
#creative common video from https://www.youtube.com/watch?v=BLaf9_jHQyQ

import ipywidgets
from IPython.display import display
import numpy as np
import base64
import cv2


rimage_widget = ipywidgets.Image(format='jpeg')

display(rimage_widget)

cap = cv2.VideoCapture('faceR.mp4')

frames = []

fpath='./cascades_cuda/haarcascade_frontalface_alt.xml'

cuImage = cv2.cuda_GpuMat()

face_detect = cv2.cuda_CascadeClassifier.create(fpath)

while(1):
    try:
        _, frame = cap.read()
       
        #fgmask = cv2.Canny(frame, 100, 100)
        frame_resized = cv2.resize(frame,(int(vwidth),int(vheight)))
        imgframe = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        #gray = cv2.cvtColor(imgframe, cv2.COLOR_BGR2GRAY)

        cuImage.upload(imgframe)
        gray = cv2.cuda.cvtColor(cuImage,cv2.COLOR_BGR2GRAY)
        faces = face_detect.detectMultiScale(gray).download()
        gray = gray.download()

        #faces = face_detect.detectMultiScale(gray, 1.3, 5)
        for (x,y,w,h) in faces[0]:
            cv2.rectangle(frame_resized,(x,y),(x+w,y+h),(255,0,0),2)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame_resized[y:y+h, x:x+w]
            roi = cv2.GaussianBlur(roi_color, (23, 23), 30)
            # impose this blurred image on original image to get final image
            frame_resized[y:y+roi.shape[0], x:x+roi.shape[1]] = roi
        rimage_widget.value = bytes(cv2.imencode('.jpg', frame_resized)[1])
      
    except Exception:
        break


cap.release()

Image(value=b'', format='jpeg')

CPU times: user 10.2 s, sys: 2.1 s, total: 12.3 s
Wall time: 25.4 s


In [6]:
from ipywidgets import Widget
Widget.close_all()

In [12]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}